In [1]:
from faker import Faker
from faker.providers import company
from uuid import uuid4
import bw2data as bd
import bw2io as bi
import random

Using environment variable BRIGHTWAY2_DIR for data directory:
/Users/chrismutel/Desktop/MF


In [2]:
fake = Faker()
fake.add_provider(company)

In [3]:
bd.projects.delete_project("MF", True)

'default'

## Set up project

In [4]:
bd.projects.set_current("MF")

In [5]:
bi.bw2setup()

Writing activities to SQLite3 database:


Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/01/2023 16:28:50
  Finished: 09/01/2023 16:28:50
  Total time elapsed: 00:00:00
  CPU %: 99.80
  Memory %: 1.23
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations



In [6]:
flows = [obj for obj, _ in zip(bd.Database("biosphere3"), range(10))]
flows

['Barium II' (kilogram, None, ('soil',)),
 'Chlormequat chloride' (kilogram, None, ('soil', 'agricultural')),
 'Lanthanum-140' (kilo Becquerel, None, ('water', 'ocean')),
 'o-Xylene' (kilogram, None, ('water', 'ground-, long-term')),
 'Caesium-134' (kilo Becquerel, None, ('water', 'ground-, long-term')),
 'VOC, volatile organic compounds' (kilogram, None, ('water',)),
 'Lanthanum-140' (kilo Becquerel, None, ('water',)),
 'Butyl acetate' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Tin ion' (kilogram, None, ('air', 'urban air close to ground')),
 'Acrylonitrile' (kilogram, None, ('water', 'ocean'))]

In [7]:
db = bd.Database("multifunctional")
db.register()

## Add multifunctional process

In [8]:
mf_node = db.new_activity(
    code=uuid4().hex,
    name=fake.company(),
    location='GLO',
    unit='kilogram',
    type="multifunctional",
)
mf_node.save()

# Add biosphere flows
for _ in range(3):
    mf_node.new_exchange(
        input=random.choice(flows),
        amount=random.random(),
        type="biosphere",
    ).save()

# Create products
for _ in range(2):
    product = db.new_activity(
        code=uuid4().hex,
        name=fake.name(),
        unit='kilogram',
        type="product",
    )
    product.save()

    mf_node.new_exchange(
        input=product,
        amount=random.random() / 2 + .5,
        type="production",
        price=random.random() * 10,
    ).save()

## Create rest of the database

In [9]:
inputs = []
   
# Add monofunctional processes
for _ in range(2):
    node = db.new_activity(
        code=uuid4().hex,
        name=fake.company(),
        location='GLO',
        unit='kilogram',
        **{"reference product": fake.company_suffix()},
    )
    node.save()
    
    for _ in range(3):
        node.new_exchange(
            input=random.choice(flows),
            amount=random.random(),
            type="biosphere",
        ).save()
    inputs.append(node)

In [10]:
for node in inputs:
    mf_node.new_exchange(
        input=node,
        amount=random.random(),
        type="technosphere",
    ).save()

In [11]:
def economic_allocation(dataset):
    total_value = sum(exc['price'] * exc['amount'] for exc in dataset.production())
    
    for product_exc in dataset.production():
        allocation_fraction = product_exc['price'] * product_exc['amount'] / total_value
        
        allocated_ds = db.new_activity(**{
            'name': dataset['name'],
            'code': "{}_{}".format(dataset['code'], product_exc.input['code']),
            'reference product': product_exc.input['name'],
            'unit': product_exc.input['unit'],
            'location': dataset['location'],
            'type': 'process',
            # Add in other desired attributes
        })
        allocated_ds.save()

        for exc in dataset.exchanges():
            if exc.input == product_exc.input:
                allocated_ds.new_exchange(input=exc.input, amount=exc['amount'], type="production").save()
            elif exc['type'] == 'production':
                continue
            else:
                allocated_ds.new_exchange(
                    input=exc.input,
                    amount=exc['amount'] * allocation_fraction,
                    type=exc['type'],
                ).save()


In [12]:
economic_allocation(mf_node)

In [13]:
for node in db:
    print(node.get('type', 'process'), node, node.get("reference product"))
    for exc in node.exchanges():
        print("\t", exc['type'], exc)
        
    print("\n")

product 'Joseph Stevens' (kilogram, GLO, None) None


multifunctional 'Johnson Ltd' (kilogram, GLO, None) None
	 biosphere Exchange: 0.9901093347062252 kilogram 'Barium II' (kilogram, None, ('soil',)) to 'Johnson Ltd' (kilogram, GLO, None)>
	 biosphere Exchange: 0.021556898058709417 kilo Becquerel 'Lanthanum-140' (kilo Becquerel, None, ('water', 'ocean')) to 'Johnson Ltd' (kilogram, GLO, None)>
	 biosphere Exchange: 0.9676784690509767 kilo Becquerel 'Lanthanum-140' (kilo Becquerel, None, ('water',)) to 'Johnson Ltd' (kilogram, GLO, None)>
	 production Exchange: 0.8155599087773919 kilogram 'Nathan Mullins' (kilogram, GLO, None) to 'Johnson Ltd' (kilogram, GLO, None)>
	 production Exchange: 0.620819254771529 kilogram 'Joseph Stevens' (kilogram, GLO, None) to 'Johnson Ltd' (kilogram, GLO, None)>
	 technosphere Exchange: 0.011069283986022138 kilogram 'Sanchez, Wilson and Williams' (kilogram, GLO, None) to 'Johnson Ltd' (kilogram, GLO, None)>
	 technosphere Exchange: 0.9466913069171184 kilo